In [1]:
import Polatis
import visa
rm = visa.ResourceManager()
rm.list_resources()

p = Polatis.Polatis()

ImportError: No module named Polatis

### Reading from the ProberConfig.conf file at initial boot

In [ ]:
configFile= [
 {'A': 'GPIB0::3::INSTR', 'P': '1::17', 'O': 'SantecTSL210H', 'Stage': 'M'},
 {'A': 'GPIB0::10::INSTR', 'P': '2,3,4::18,19,20', 'O': 'HP8163A', 'Stage': 'M'},
 {'A': 'GPIB0::7::INSTR', 'P': '5,6,7::21,22,23', 'O': 'Wiltron68145B', 'Stage': 'M'},
 {'A': 'GPIB0::23::INSTR', 'P': '15::30', 'O': 'AnritsuMS2667C', 'Stage': 'M'}
]

stages = {}

In [ ]:
integration = SwitchIntegration(configFile, stages, p)

In [ ]:
holder = integration.switch_book

In [ ]:
holder

In [ ]:
p.get_zip_connections()

In [ ]:
p.quick_connect(15, 17)

In [ ]:
p.get_zip_connections()

In [ ]:
def _hasChangedOutput(entry, pair):
    
    return pair[0] == entry['ingress'] and pair[1] != entry['egress']

def _hasLostOutput(entry, pair):

    return entry['ingress'] not in [i[0] for i in actual]

actual = p.get_zip_connections()
for entry in holder:
    for pair in actual:
        if _hasChangedOutput(entry, pair): # then this device changed output
            entry['egress'] = pair[1]
        elif _hasLostOutput(entry, pair): # then this device changed output
            entry['egress'] = ''

In [ ]:
def _updateSwitchBook(self):
    '''Updates the global switch_book following a new connection being made'''
    
    actual = self._p.get_zip_connections()
    for entry in self.switch_book:
        for pair in actual:
            if self._hasChangedOutput(entry, pair): # then this device changed output
                entry['egress'] = pair[1]
            elif self._hasLostOutput(entry, pair): # then this device changed output
                entry['egress'] = ''

## Internal functions

In [ ]:
# import Polatis

class SwitchIntegration(object):
    '''The intention of this class if to provide an implementation of the Polatis switch.
    The key features of the class are get_switch_state(), make_new_connection() and switch_book object.'''

    def __init__(self, configFile, stages, polatis):

        self._p = polatis

        self.switch_book = self._readIn(configFile)
        self._connect()
        # self._constructComparable() # I dont think I need this anymore because we've moved to single association entries 
        self._integrateStages(stages)

    def get_switch_state(self):
        '''
        Queries all current connections.

        :returns: String of all connections between devices
        '''
        
        formatted_output = ""
        for entry in self.switch_book:
            formatted_output += "{:15} {:20}{:3} {:5} {:10}\n".format(
            'Device',entry['device'],entry['ingress'],"-->",entry['egress']
            )
        return formatted_output

#     def _formatPort(self, entry):
#         '''internal use'''
        
#         return toReturn

#     def _formatInactivePort(self, entry):
#         '''internal use'''
#         toReturn = "{:15} {:20} Ports: {:5}\n".format(
#             'Not in use:',entry['device'], entry['ingress'].split(',')
#         )
#         return toReturn

##############################################################################
    def make_new_connection(self, ingress, egress): # stays the same
        '''
        User specified connection between device ingress port and output port.

        :param ingress: Specified input port
        :type ingress: Int
        :param egress: Specified output port
        :type egress: Int
        '''
        
        self._p.quick_connect(ingress, egress)
        self._updateSwitchBook()
        
##############################################################################
    def _readIn(self, configFile):# Modified
        '''Reads in the configFile info'''
        switch_book_init = []
        for entry in configFile:

            ports = entry['P'].split('::')
            ingress = ports[0].split(',')
            egress = ports[1].split(',')
            ports = zip(ingress, egress)

            for pair in ports:
                new_entry = {}
                new_entry['ingress'] = pair[0]
                new_entry['egress'] = pair[1]
                new_entry['device'] = entry['O']
                switch_book_init.append(new_entry)
                
        return switch_book_init
            
##############################################################################
    def _connect(self):# Modified  
        '''makes connections between ports for SwitchIntegration.__init__()'''
        for entry in self.switch_book:
            # Call function for making connections
            self._p.quick_connect(
                ingress=entry['ingress'],
                egress=entry['egress']
            )
##############################################################################
# potentially deprecated
#     def _constructComparable(self):
#         '''Constructs tuples for connections key in switch book'''
        
#         for entry in self.switch_book:
#             new_value = []
#             #Multi Association
#             if len(entry['ingress']) != 1:
#                 multi = self._multiConnHandler(entry)
#                 for item in multi:
#                     new_value.append(item)
#             else:
#                 #Single Association
#                 new_value.append((entry['ingress'],entry['egress']))
                
#             entry['connections'] = new_value
            
#         self._cleanUp()

#     def _cleanUp(self):
#         '''cleans up the switch_book after boot'''
#         for entry in self.switch_book:
#             entry.pop('egress', None)

#     def _multiConnHandler(self, entry):
#         '''helper for __constructComparable()'''
#         ingress = [i for i in entry['ingress'].split(',')]
#         egress = [i for i in entry['egress'].split(',')]
        
#         return zip(ingress, egress)
##############################################################################
    def _integrateStages(self, stages): # Stays the same, keep it 
        '''Integrates the stages dictionary after boot'''
        for actualObject in zip(stages.keys(), stages.values()):
            for entry in self.switch_book:
                if entry['device'] == str(actualObject[1]):
                    entry['ref'] = actualObject[0]
##############################################################################
    def _updateSwitchBook(self): # modified
        '''Updates the global switch_book following a new connection being made'''
        actual = self._p.get_zip_connections()
        for entry in self.switch_book:
            for pair in actual:
                if self._hasChangedOutput(entry, pair): # then this device changed output
                    entry['egress'] = pair[1]
                elif self._hasLostOutput(entry, pair, actual): # then this device changed output
                    entry['egress'] = ''
                    
    def _hasChangedOutput(self, entry, pair):
        '''helper for _updateSwitchBook'''
        return pair[0] == entry['ingress'] and pair[1] != entry['egress']

    def _hasLostOutput(self, entry, pair, actual):
        '''helper for _updateSwitchBook'''
        return entry['ingress'] not in [i[0] for i in actual]

In [ ]:
integration = SwitchIntegration(configFile=configFile,stages=stages,polatis=p)

In [ ]:
print integration.get_switch_state()

### Making connection between Antri 15 --> 29

In [ ]:
integration.make_new_connection(15,29)
print integration.get_switch_state()

### Making a connection between Antri 15 --> 17
- so Santec should read 1 --> 
- Antri should read 15 --> 17

In [ ]:
integration.make_new_connection(15,17)
print integration.get_switch_state()

### Regaining connection between 1 --> something

In [ ]:
integration.make_new_connection(1,25)
print integration.get_switch_state()

### Taking away 1 --> 25 connection and then giving 17 to 1

In [ ]:
integration.make_new_connection(2,25)
integration.make_new_connection(1,17)
print integration.get_switch_state()

In [ ]:
# import Polatis

class SwitchIntegration(object):
    '''The intention of this class if to provide an implementation of the Polatis switch.
    The key features of the class are get_switch_state(), make_new_connection() and switch_book object.'''

    def __init__(self, configFile, stages, polatis):

        self._p = polatis

        self.switch_book = self._readIn(configFile)
        self._connect()
        self._integrateStages(stages)

    def get_switch_state(self):
        '''
        Queries all current connections.

        :returns: String of all connections between devices
        '''
        
        formatted_output = ""
        for entry in self.switch_book:
            formatted_output += "{:15} {:20}{:3} {:5} {:10}\n".format(
            'Device',entry['device'],entry['ingress'],"-->",entry['egress']
            )
        return formatted_output

    def make_new_connection(self, ingress, egress): 
        '''
        User specified connection between device ingress port and output port.

        :param ingress: Specified input port
        :type ingress: Int
        :param egress: Specified output port
        :type egress: Int
        '''
        
        self._p.quick_connect(ingress, egress)
        self._updateSwitchBook()
        

    def _readIn(self, configFile):
        '''Reads in the configFile info'''
        switch_book_init = []
        for entry in configFile:

            ports = entry['P'].split('::')
            ingress = ports[0].split(',')
            egress = ports[1].split(',')
            ports = zip(ingress, egress)

            for pair in ports:
                new_entry = {}
                new_entry['ingress'] = pair[0]
                new_entry['egress'] = pair[1]
                new_entry['device'] = entry['O']
                switch_book_init.append(new_entry)
                
        return switch_book_init
            
    def _connect(self):
        '''makes connections between ports for SwitchIntegration.__init__()'''
        for entry in self.switch_book:
            # Call function for making connections
            self._p.quick_connect(
                ingress=entry['ingress'],
                egress=entry['egress']
            )

    def _integrateStages(self, stages):  
        '''Integrates the stages dictionary after boot'''
        for actualObject in zip(stages.keys(), stages.values()):
            for entry in self.switch_book:
                if entry['device'] == str(actualObject[1]):
                    entry['ref'] = actualObject[0]

    def _updateSwitchBook(self):
        '''Updates the global switch_book following a new connection being made'''
        actual = self._p.get_zip_connections()
        for entry in self.switch_book:
            for pair in actual:
                if self._hasChangedOutput(entry, pair): # then this device changed output
                    entry['egress'] = pair[1]
                elif self._hasLostOutput(entry, pair, actual): # then this device changed output
                    entry['egress'] = ''
                    
    def _hasChangedOutput(self, entry, pair):
        '''helper for _updateSwitchBook'''
        return pair[0] == entry['ingress'] and pair[1] != entry['egress']

    def _hasLostOutput(self, entry, pair, actual):
        '''helper for _updateSwitchBook'''
        return entry['ingress'] not in [i[0] for i in actual]